In [2]:
import math
import os
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from dotenv import load_dotenv
from tqdm.auto import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    GenerationConfig,
    set_seed,
)

import src.utils.data as data_utils
import src.utils.io as io_utils
import src.utils.models as model_utils

In [3]:
load_dotenv()

warnings.filterwarnings("ignore")
%matplotlib inline
%load_ext autoreload
%autoreload 2

# EXTERNAL = Path(os.getenv("EXTERNAL_STORAGE_DIR"))
ROOT = io_utils.repo_root()
SPLIT_DIR = ROOT / "data/splits"
CONFIG_DIR = ROOT / "config"
METRIC_DIR = ROOT / "metrics"
RANDOM_STATE = 42
set_seed(RANDOM_STATE)

In [4]:
ROOT

PosixPath('/content/llm-news')

In [5]:
VAL_IDS_PATH = io_utils.load_yaml(CONFIG_DIR / "dataset.ids.yml")["splits_ids"][
    "val_ids"
]
val_ids = pd.read_csv(ROOT / VAL_IDS_PATH, header=None)

In [6]:
raw_val = load_dataset("IlyaGusev/gazeta")["validation"].to_pandas()

print("raw val shape:", raw_val.shape)
raw_val.head()

README.md: 0.00B [00:00, ?B/s]

0000.parquet:   0%|          | 0.00/252M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/22.7M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/27.8M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/30.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

raw val shape: (6369, 5)


,text,summary,title,date,url
0,"В 2020 году инфляция в России составит 3,5-4%,...",В уходящем году инфляция в России находится на...,Дорогой 2020-й: какие продукты подскочат в цене,2020-01-01 10:29:58,https://www.gazeta.ru/business/2019/12/26/1288...
1,Глава Белого дома Дональд Трамп выразил надежд...,Мировая общественность призвала лидера КНДР Ки...,Подарок от Ким Чен Ына: Трамп ответил на новые...,2020-01-01 11:16:44,https://www.gazeta.ru/army/2020/01/01/12894224...
2,Председатель Союза еврейских религиозных общин...,Главный раввин Польши Михаил Шудрих раскритико...,Это манипуляция: раввин Польши ответил Путину ...,2020-01-01 12:39:08,https://www.gazeta.ru/politics/2020/01/01_a_12...
3,Первой песней наступившего 2020 года на Первом...,Народная артистка СССР София Ротару выступила ...,Первая после Путина: Ротару выступила на Новый...,2020-01-01 13:36:41,https://www.gazeta.ru/culture/2020/01/01/a_128...
4,"Выражение «глубинное государство», пришедшее и...",1 января 1895 года родился самый могущественны...,"Пресли, Леннон, Трумэн: против кого работал Эд...",2020-01-01 14:20:16,https://www.gazeta.ru/politics/2019/12/30_a_12...


In [7]:
val = raw_val.loc[val_ids.squeeze(), ["title", "text", "summary"]]
for col in val.columns:
    val[col] = data_utils.clean(val[col])
val.head(2)

,title,text,summary
0,Дорогой 2020-й: какие продукты подскочат в цене,"В 2020 году инфляция в России составит 3,5-4%,...",В уходящем году инфляция в России находится на...
1,Подарок от Ким Чен Ына: Трамп ответил на новые...,Глава Белого дома Дональд Трамп выразил надежд...,Мировая общественность призвала лидера КНДР Ки...


In [8]:
MODEL_CFG_PATH = CONFIG_DIR / "models.params.yml"
model_cfg = None
if torch.cuda.is_available():
    model_cfg = io_utils.load_yaml(MODEL_CFG_PATH)["cuda_model"]
else:
    model_cfg = io_utils.load_yaml(MODEL_CFG_PATH)["cpu_model"]

model_cfg

{'device': 'cuda',
 'model_id': 'Qwen/Qwen2-7B-Instruct',
 'n_eval': 500,
 'use_4bit': True,
 'device_map': 'auto'}

In [9]:
device = model_cfg["device"]
model_id = model_cfg["model_id"]
n_eval = model_cfg["n_eval"]
use_4bit = model_cfg["use_4bit"]
device_map = model_cfg["device_map"]
torch_dtype = (
    torch.bfloat16
    if device == "cuda" and torch.cuda.is_bf16_supported()
    else (torch.float16 if device == "cuda" else torch.float32)
)
if n_eval is None:
    subset_val = val
else:
    subset_val = val.sample(n=min(n_eval, val.shape[0]), random_state=RANDOM_STATE)

subset_val.head(2)

,title,text,summary
6238,Украина снова закрывает границы для иностранцев,Украина снова закрывает границу для въезда ино...,Возобновление роста случаев коронавирусной инф...
217,Много пьют: Горбачев призвал отменить новогодн...,Россиянам не нужны «длинные выходные» под Новы...,Экс-президент СССР Михаил Горбачев призвал отм...


In [10]:
quantization_config = None
if use_4bit:
    try:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch_dtype,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
        )
    except Exception as e:
        print("bitsandbytes не готов, продолжаем без 4-бит:", e)
        quantization_config = None

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=(None if quantization_config else torch_dtype),
    device_map=device_map,
    quantization_config=quantization_config,
)

# 1) для decoder-only нужно левое выравнивание
tokenizer.padding_side = "left"

# 2) при обрезке важно сохранять «конец» промпта (там префикс ассистента)
tokenizer.truncation_side = "left"

# 3) если у модели нет отдельного PAD — используем EOS
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token = tokenizer.eos_token

model.config.pad_token_id = tokenizer.pad_token_id
if getattr(model, "generation_config", None) is not None:
    model.generation_config.pad_token_id = tokenizer.pad_token_id

if device != "cuda":
    model.to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [12]:
SYSTEM_PROMPT = (
    "Ты помощник по резюмированию русскоязычных новостей. "
    "Сделай краткое, нейтральное резюме исходного текста (3–5 предложений). "
    "Не добавляй фактов, которых нет в тексте."
)

GEN_EVAL = GenerationConfig(
    max_new_tokens=160,
    do_sample=False,
)

# 1) для decoder-only нужно левое выравнивание
tokenizer.padding_side = "left"

# 2) при обрезке важно сохранять «конец» промпта (там префикс ассистента)
tokenizer.truncation_side = "left"

# 3) если у модели нет отдельного PAD — используем EOS
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token = tokenizer.eos_token


MAX_INPUT_TOKENS = model_utils.get_max_input_tokens(tokenizer, GEN_EVAL)


def build_chat(text: str):
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {
            "role": "user",
            "content": f"Задача: кратко резюмируй.\n\nТекст статьи:\n{text}",
        },
    ]


def generate_batch(
    texts: list[str], batch_size: int = 1, show_progress: bool = True
) -> list[str]:
    out = []
    model.eval()

    it = range(0, len(texts), batch_size)
    if show_progress:
        it = tqdm(
            it, total=math.ceil(len(texts) / batch_size), desc="Generating", leave=False
        )

    # на всякий случай — паддинг токен
    if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
        tokenizer.pad_token = tokenizer.eos_token

    for i in it:
        chunk = texts[i : i + batch_size]

        # 1) шаблон → строки
        prompts = [
            tokenizer.apply_chat_template(
                build_chat(t), tokenize=False, add_generation_prompt=True
            )
            for t in chunk
        ]

        # 2) строки → тензоры (BatchEncoding / dict)
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            pad_to_multiple_of=8,
            max_length=MAX_INPUT_TOKENS,
        ).to(device)

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                generation_config=GEN_EVAL,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=getattr(tokenizer, "eos_token_id", None),
            )

        # вырезаем только ответ
        gen_ids = output_ids[:, inputs["input_ids"].shape[1] :]
        decoded = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
        cleaned = [d.strip() for d in decoded]
        out.extend(cleaned)

    return out


BATCH = 1 if device != "cuda" else 6

preds_llm = generate_batch(
    subset_val["text"].tolist(), batch_size=BATCH, show_progress=True
)
refs_llm = subset_val["summary"].tolist()
len(preds_llm), len(refs_llm)

Generating:   0%|          | 0/84 [00:00<?, ?it/s]

(500, 500)

In [13]:
preds_llm[:2]

['Украина вновь ввела ограничения на въезд иностранцев из-за роста случаев коронавируса, вступивших в силу с 29 августа по 28 сентября. Ограничения не касаются водителей грузового транспорта, инструкторов из стран НАТО, деятелей культуры приглашенных культурными учреждениями и некоторых других категорий. Президент Украины Владимир Зеленский согласился с необходимостью продления карантина до 1 ноября, однако подчеркнул, что режим должен быть адаптивным и необременительным для бизнеса. Лидер украинской партии «Батьки',
 'Бывший президент СССР Михаил Горбачев в интервью изданию Ura.ru выразил мнение, что российские выходные дни, концентрированные дважды в год, приводят к бездействию и злоупотреблению алкоголем среди населения. Горбачев предложил распределить выходные дни более равномерно по календарю. Он также напомнил о том, что в 1980-х годах, когда он был в должности, "длинных выходных" практически не существовало, что положительно сказывалось на обществе. Эксперт Г']

In [14]:
refs_llm[:2]

['Возобновление роста случаев коронавирусной инфекции вынудило правительство снова закрыть границы для иностранцев. С 29 августа по 28 сентября на Украину смогут попасть только иностранцы, имеющие разрешение на проживание в стране, а также члены международных и гуманитарных миссий, транзитные пассажиры и ряд других категорий. Всего в стране COVID-19 болеют 54 277 человек.',
 'Экс-президент СССР Михаил Горбачев призвал отменить новогодние каникулы и другие «длинные выходные» в России. По словам политика, народ в это время «бездельничает», а многие уходят в продолжительный запой. Он добавил, что выходные дни января и мая имеет смысл распределить более равномерно «по календарю».']

In [15]:
# rouge_scores = data_utils.get_rouge_f1(preds_llm, refs_llm)

# rouge_scores

In [16]:
scores = data_utils.get_all_scores(preds_llm, refs_llm, device=device)
scores

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

{'rouge1': np.float64(0.252879736610214),
 'rouge2': np.float64(0.09644614650765541),
 'rougeL': np.float64(0.2456477749332227),
 'rougeLsum': np.float64(0.24771076457261973),
 'bertscore_precision': 0.8671674896478653,
 'bertscore_recall': 0.8825865497589112,
 'bertscore_f1': 0.8747395805120468,
 'avg_len_pred': 62.528,
 'avg_len_ref': 45.34,
 'len_ratio_pred_to_ref': 1.3790913101014555}

In [19]:
Path(METRIC_DIR).mkdir(parents=True, exist_ok=True)

df_metrics = pd.DataFrame(
    [
        {
            "system": "extractive_lead3",
            "split": "validation_full",
            "rouge1": scores.get("rouge1", 0.0),
            "rouge2": scores.get("rouge2", 0.0),
            "rougeL": scores.get("rougeL", 0.0),
            "rougeLsum": scores.get("rougeLsum", 0.0),
            "bertscore_precision": scores.get("bertscore_precision", 0.0),
            "bertscore_recall": scores.get("bertscore_recall", 0.0),
            "bertscore_f1": scores.get("bertscore_f1", 0.0),
            "avg_len_pred": scores.get("avg_len_pred", 0.0),
            "avg_len_ref": scores.get("avg_len_ref", 0.0),
            "len_ratio_pred_to_ref": scores.get("len_ratio_pred_to_ref", 0.0),
            "k": None,
            "n_examples": n_eval,
        }
    ]
)
df_metrics.to_csv(
    METRIC_DIR / f"llm_zero_shot_validation_{device}_{n_eval}.csv", index=False
)

df_sampels = pd.DataFrame(
    [
        {
            "title": subset_val["title"].head(3) if "title" in subset_val else [""] * 3,
            "reference": refs_llm[:3],
            "prediction": preds_llm[:3],
        }
    ]
)
df_sampels.to_csv(
    METRIC_DIR / f"llm_zero_shot_examples_{device}.tsv", sep="\t", index=False
)

In [1]:
!nvidia-smi

import torch

print("torch:", torch.__version__, "| CUDA доступна:", torch.cuda.is_available())

# ----------------------------------------------------------------------------------

from google.colab import drive

drive.mount("/content/drive", force_remount=True)

# ----------------------------------------------------------------------------------

import os

BASE = "/content/drive/MyDrive/llm-news"
for sub in ["models", "metrics", "hf_cache"]:
    os.makedirs(os.path.join(BASE, sub), exist_ok=True)

print("Созданы/проверены папки:", os.listdir(BASE))

# ----------------------------------------------------------------------------------

import subprocess
import sys

REPO_URL = "https://github.com/mdayssi/llm-news-summarizer-ru.git"
REPO_DIR = "/content/llm-news"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    print("Репозиторий уже есть:", REPO_DIR)


%cd {REPO_DIR}
!git rev-parse --short HEAD


# ----------------------------------------------------------------------------------

from pathlib import Path

env_path = Path(REPO_DIR) / ".env"
kv = {
    "EXTERNAL_MODELS_DIR": "/content/drive/MyDrive/llm-news/models",
    "EXTERNAL_METRICS_DIR": "/content/drive/MyDrive/llm-news/metrics_big",
    "EXTERNAL_CACHE_DIR": "/content/drive/MyDrive/llm-news/hf_cache",
}
text = "\n".join([f"{k}={v}" for k, v in kv.items()]) + "\n"
env_path.write_text(text, encoding="utf-8")

print(".env создано:")
print(env_path.read_text())


# ----------------------------------------------------------------------------------
%pip -q install --upgrade \
  evaluate rouge-score bert_score\
  razdel bitsandbytes\
  python-dotenv pyyaml \

import accelerate
import bert_score
import bitsandbytes
import datasets
import dotenv
import evaluate
import razdel
import rouge_score
import sentencepiece
import torch
import tqdm
import transformers
import yaml

print("torch:", torch.__version__, "| cuda avail:", torch.cuda.is_available())
print("transformers:", transformers.__version__)
print("datasets:", datasets.__version__)
print("evaluate:", evaluate.__version__)

# ----------------------------------------------------------------------------------
import sys

repo_src = "/content/llm-news/src"
if repo_src not in sys.path:
    sys.path.insert(0, repo_src)
print("sys.path ok")

Sun Aug 17 17:55:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             46W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
# ----------------------------------------------------------------------------------
# !git config --global user.email "dasha.morgalenko@gmail.com"
# !git config --global user.name "mdayssi"
# !git remote set-url origin https://$GITHUB_TOKEN@github.com/mdayssi/llm-news-summarizer-ru.git

# !git branch --show-current
# !git remote -v
# !git fetch origin
# !git pull --rebase origin main


# !git status
# !git add .
# !git commit -m "add metrics zero-shot 7B on cuda"
# !git push origin main